# UK Cyber Fraud Support Chatbot Demo
## Fine-tuned LLaMA 2 for Cyber Fraud Support

In [19]:
# Install required packages
!pip install gradio torch transformers peft bitsandbytes accelerate huggingface_hub -q

In [20]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from huggingface_hub import login
import warnings
warnings.filterwarnings('ignore')

# Login to Hugging Face
login()

In [21]:
# Model configuration
MODEL_NAME = "NousResearch/Llama-2-7b-chat-hf"
ADAPTER_MODEL_NAME = "misee/uk-fraud-chatbot-llama2"

# Load model with 4-bit quantization
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

print("Loading base model...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

model = PeftModel.from_pretrained(base_model, ADAPTER_MODEL_NAME)
print("Model loaded successfully!")

Loading base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded successfully!


In [22]:
def chat_with_model(message, history, system_message, max_tokens, temperature, top_p):
    """Generate response from the fine-tuned model"""
    try:
        # Format for LLaMA 2 chat
        formatted_prompt = f"<s>[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{message} [/INST]"

        inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                do_sample=True,
                temperature=temperature,
                top_p=top_p,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.1,
                no_repeat_ngram_size=3,
            )

        full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = full_response.split("[/INST]")[-1].strip()

        # Clean up incomplete sentences at the end
        sentences = response.split('.')
        if len(sentences) > 1 and len(sentences[-1].strip()) < 10:
            # Remove last incomplete sentence if it's very short
            response = '.'.join(sentences[:-1]) + '.'

        return response

    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
# Create Gradio interface
demo = gr.ChatInterface(
    chat_with_model,
    title="UK Fraud Support Chatbot Demo",
    description="Fine-tuned LLaMA 2 for fraud victim support",
    additional_inputs=[
        gr.Textbox(
            value="You are a specialised UK fraud victim support assistant. Provide practical guidance for cyber fraud assistance in the UK.",
            label="System message"
        ),
        gr.Slider(minimum=100, maximum=800, value=400, label="Max tokens"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.7, label="Temperature"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.9, label="Top-p"),
    ],
)

# Launch with public sharing
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://09d125adf8e538f47d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
